In [1]:
#importing libararies
import pandas as pd
import numpy as np
import os
from __future__ import print_function
from mailmerge import MailMerge
from datetime import date
import docx
import import_ipynb
import glob
import tkinter
from tkinter import filedialog
import warnings
warnings.filterwarnings("ignore")


def format_sheet(file_path:str,template_path:str,sheet_num_template=0,run=0):
    """
    file_path: the file you want to design, must bx xlsx
    template_path: the teplate of the design, must bx xlsx
        """
    # format sheet
    xlsx=pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    for i in range(len(xlsx.book.worksheets)):
        new_sheet=xlsx.book.worksheets[run]
        template=pd.ExcelWriter(template_path,engine='openpyxl', mode='a')
        default_sheet=template.book.worksheets[sheet_num_template]
        from copy import copy
        new_sheet.sheet_view.rightToLeft=True
        for row in default_sheet.rows:
            for cell in row:
                new_cell=new_sheet.cell(row=cell.row, column=cell.col_idx)
                if cell.has_style and new_cell.value != None:
                    new_cell.font=copy(cell.font)
                    new_cell.border=copy(cell.border)
                    new_cell.fill=copy(cell.fill)
                    new_cell.number_format=copy(cell.number_format)
                    new_cell.protection=copy(cell.protection)
                    new_cell.alignment=copy(cell.alignment)
        from openpyxl.utils import get_column_letter
        for i in range(default_sheet.max_column):
            new_sheet.column_dimensions[get_column_letter(i + 1)].width=default_sheet.column_dimensions[
                get_column_letter(i + 1)].width
    
    xlsx.save()
def change_names_and_order(file_path:str,df:pd.DataFrame):
    """
    name of columns in names must be "old_name", "new_name"
        """
    match_names=pd.read_excel(file_path)
    match_names.index=match_names.old_name
    dict_names=match_names.to_dict()['new_name']
    ls_names=match_names['new_name'].tolist()
    df.rename(columns=dict_names, inplace=True)
    df=df[ls_names]
    return df

In [2]:

tkinter.Tk().withdraw() # prevents an empty tkinter window from appearing

transction_folder_path = filedialog.askdirectory()
permanant_folder_path = filedialog.askdirectory()

In [36]:

#reading the check file
filepath_check = transction_folder_path+"\*RIA*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    check_sent = pd.read_excel(textfile,sheet_name="Sent TX",skiprows=2)
    check_paid = pd.read_excel(textfile,sheet_name="Paid TX",skiprows=2)


#reading the risk country file
filepath_check = permanant_folder_path+"\*מדינות*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    rc = pd.read_excel(textfile)

#reading the rates file
filepath_check = permanant_folder_path+"\*שערים*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    rates = pd.read_excel(textfile ,sheet_name="CUR CONV",usecols="F:G",skiprows=1)
    

#reading the report number generator file
filepath_check = transction_folder_path+"\*מחולל*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    report_num = pd.read_excel(textfile)
    
#reading the report content file
filepath_check = transction_folder_path+"\*content*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    content = pd.read_excel(textfile)  
    
#reading the report branches file
filepath_check = transction_folder_path+"\*Branch*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    Branches = pd.read_excel(textfile)  

In [28]:
#getting the data ready
#naming rates columns
rates.columns=["curr","rate"]

#adding the rates value
check_sent = check_sent[check_sent['Canceled Time'].isna()]
check_sent= check_sent.merge(rates, how = 'left', left_on = 'Currency', right_on = 'curr')
check_sent['nis sum'] = check_sent['rate'] * check_sent['Amount']
check_sent = check_sent.dropna(subset = 'nis sum')
check_sent['nis sum'] = check_sent['nis sum'].astype(int)
#lower the country cases:
check_sent['ID Issued BY Country'] = check_sent['ID Issued BY Country'].str.lower()
check_sent['Ben Country'] = check_sent['Ben Country'].str.lower()
#check['Source country'] = check['Source country'].str.lower()
rc['Alpha-2 code'] = rc['Alpha-2 code'].str.lower()
check_sent = check_sent.fillna('')

#risk check
#check_sent = check.merge(rc[['English short name (using title case)','סיכון/ללא דיווח רגיל']] , left_on = 'Source country', right_on = 'English short name (using title case)',how = 'left',suffixes = ('','_source'))
check_sent = check_sent.merge(rc[['Alpha-2 code','סיכון/ללא דיווח רגיל']] , left_on = 'Ben Country', right_on = 'Alpha-2 code',how = 'left',suffixes = ('','_dest'))
check_sent = check_sent.merge(rc[['Alpha-2 code','סיכון/ללא דיווח רגיל']] , left_on = 'ID Issued BY Country', right_on = 'Alpha-2 code',how = 'left',suffixes = ('','_issuance'))


#date coolums as dat and time and a columns with the month number
check_sent['TX Date']= pd.to_datetime(check_sent['TX Date'],dayfirst=True)
check_sent['month'] = pd.DatetimeIndex(check_sent['TX Date']).month


In [29]:

#outbound or inbound
#outbound = check[check['Type'] == 'SEND']
#inbound = check[check['Type'] == 'PAYOUT']

#getting the names ready
check_sent['Sender'] = (check_sent['Cust First Name'] + ' ' + check_sent['Cust Last Name 1']).str.lower()
check_sent['Receiver'] = (check_sent['Ben First Name'] + ' ' + check_sent['Ben LastName 1']).str.lower()

#check risk 
check_risk_sent = check_sent[ (check_sent['סיכון/ללא דיווח רגיל'].notna()) | (check_sent['סיכון/ללא דיווח רגיל_issuance'].notna())]

#building the monitoring df with sum per client, deal distinct count and under 50k deal count
sent_monitoring = check_sent.groupby(['Sender','month']).agg({'nis sum':'sum','Order No':'nunique'})
sent_monitoring = sent_monitoring.reset_index()

#sorting the values by sum
sent_monitoring = sent_monitoring.sort_values('nis sum',ascending = False)
sent_monitoring_excel = sent_monitoring
#filter only client over 47K
sent_monitoring = sent_monitoring[sent_monitoring['nis sum'] >= 47000]

#adding a column named status based on unique count of deal number
sent_monitoring = sent_monitoring[((sent_monitoring['Order No'] > 1) & (sent_monitoring['nis sum'] >= 50000))|(sent_monitoring['nis sum'] < 50000)]

#marking dirdos transction as d in a new column named dirdos
sent_monitoring['type'] = np.where((sent_monitoring['nis sum']>= 47000) & (sent_monitoring['nis sum']<50000) , "d",'p')
sent_monitoring=sent_monitoring.reset_index()

#out bound risk
#check_sent_risk = check_sent[(check_sent['סיכון/ללא דיווח רגיל'].notna()) | (check_sent['סיכון/ללא דיווח רגיל_dest'].notna()) | (check_sent['סיכון/ללא דיווח רגיל_issuance'].notna())]

#outbound tisk monitorng df
sent_risk_monitoring = check_risk_sent.groupby(['Sender','month']).agg({'nis sum':'sum','Order No':'nunique'})

#sorting the values by sum
sent_risk_monitoring = sent_risk_monitoring.sort_values('nis sum',ascending = False)

#filtering to report tr
sent_risk_monitoring = sent_risk_monitoring[sent_risk_monitoring['nis sum'] >= 4700]
sent_risk_monitoring = sent_risk_monitoring[((sent_risk_monitoring['Order No'] > 1) & (sent_risk_monitoring['nis sum'] >= 5000)) | (sent_risk_monitoring['nis sum'] < 5000)]


#adding the type of the report pr-pitzul dr-dirdos (risk)
sent_risk_monitoring['type'] =  np.where((sent_risk_monitoring['nis sum'] >= 5000), "pr" , "dr")
sent_risk_monitoring = sent_risk_monitoring.reset_index()

try:
    report = sent_risk_monitoring[['type','Sender']].merge(sent_monitoring[['type','Sender']], on = 'Sender',how = 'outer',suffixes = ('_risk',''))
except KeyError:
    report = sent_risk_monitoring[['type','Sender']]
    print('no no risk reports')
    
#building the monitoring df with sum per client and deal distinct count
check_sent_ow = check_sent.groupby(['Receiver','month']).agg({'nis sum':'sum','Order No':'nunique','Sender':'nunique'})

#sorting the values by sum
check_sent_ow = check_sent_ow.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
check_sent_ow = check_sent_ow[(check_sent_ow['Order No'] > 1) & (check_sent_ow['Sender'] > 1) & (check_sent_ow['nis sum'] >= 50000)]
check_sent_ow = check_sent_ow.reset_index()

#creating a list of the shared owner report clients name
check_sent_ow = check_sent_ow.merge(check_sent[['Sender','Receiver']], on = 'Receiver', how = 'left',suffixes = ('_count','') )
check_sent_ow = check_sent_ow.merge(report, on ='Sender' , how = 'left', suffixes = ('','_report'))


try:
    check_sent_ow = check_sent_ow[ (check_sent_ow['type'].isna()) & (check_sent_ow['type_risk'].isna())]
except KeyError:
    check_sent_ow =  check_sent_ow[(check_sent_ow['type_risk'].isna())]
        
check_sent_ow['type_m'] = 'm'
check_sent_ow = check_sent_ow.drop_duplicates('Receiver')
try:
    report = check_sent_ow[['Sender','type_m']].merge(report, on = 'Sender', how = 'outer')
except KeyError:
    report = report
    print('no shared owner reports')    
    
#building the monitoring df with sum per client and deal distinct count risk
check_sent_ow_risk = check_risk_sent.groupby(['Receiver','month']).agg({'nis sum':'sum','Order No':'nunique','Sender':'nunique'})

#sorting the values by sum
check_sent_ow_risk = check_sent_ow_risk.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
check_sent_ow_risk = check_sent_ow_risk[(check_sent_ow_risk['Order No'] > 1) & (check_sent_ow_risk['Sender'] > 1) & (check_sent_ow_risk['nis sum'] >= 5000)]
check_sent_ow_risk = check_sent_ow_risk.reset_index()

#creating a list of the shared owner report clients name____not ready yet!!!!
check_sent_ow_risk = check_sent_ow_risk.merge(check_risk_sent[['Sender','Receiver']], on = 'Receiver', how = 'left',suffixes = ('_count','') )
check_sent_ow_risk = check_sent_ow_risk.merge(report, on ='Sender' , how = 'left')

try:
    check_sent_ow_risk = check_sent_ow_risk[(check_sent_ow_risk['type'].isna()) & (check_sent_ow_risk['type_risk'].isna()) & (check_sent_ow_risk['type_m'].isna())]
except KeyError:
    check_sent_ow_risk =  check_sent_ow_risk

check_sent_ow_risk['type_m_risk'] = 'mr'
check_sent_ow_risk = check_sent_ow_risk.drop_duplicates('Receiver')

#merging the risk ow with regular report
try:
    report = check_sent_ow_risk[['Sender','type_m_risk']].merge(report, on = 'Sender', how = 'outer')
except KeyError:
    report = report

#creating the details to report inbound and outbound
#outbound
report = report.merge(check_sent[['Sender','Ben Country','Receiver','Cust DOB','Cust City','Cust Address','Cust ID NO','ID Issued BY Country']],on = 'Sender',how = 'left')
report = report.drop_duplicates('Sender')
report = report.fillna('')
report['final type'] = report['type']+report['type_risk']+report['type_m'] + report['type_m_risk']

full_report = report.fillna('')

#importing the report number needed
report_id = (report_num.iloc[1,5] + 1).astype(int)
full_report['report_id'] = report_id + range(len(full_report.index))
full_report['report_name'] = report_num.iloc[3,1].rsplit('-',1)[0]

#checking the risk type
risk_country4report = full_report.merge(rc[['Alpha-2 code','עברית','סיכון/ללא דיווח רגיל']], left_on = 'ID Issued BY Country', right_on ='Alpha-2 code', how = 'left')
risk_country4report = risk_country4report.merge(rc[['Alpha-2 code','עברית','סיכון/ללא דיווח רגיל']], left_on = 'Ben Country', right_on ='Alpha-2 code', how = 'left',suffixes = ('','_dest'))

#creating a column with the hebrew name of the country for risk report
risk_country4report['risk_content'] = np.where((risk_country4report['סיכון/ללא דיווח רגיל'].notna()) & ((risk_country4report['final type'] != 'm') | (risk_country4report['final type'] != 'p') | (risk_country4report['final type'] != 'd')) ,risk_country4report['עברית'],risk_country4report['עברית_dest'])

#merging the report content
risk_country4report = risk_country4report.merge(content, left_on = 'final type' , right_on = 'type', how = 'left')
risk_country4report['risk_full_content'] ='הכספים נשלחו ל'+risk_country4report['risk_content'] + ' ,מדינה המוגדרת בסיכון גבוה.'

risk_country4report['risk_full_content'] =np.where((risk_country4report['final type'] == 'mr')|(risk_country4report['final type'] == 'pr')|(risk_country4report['final type'] == 'dr')|(risk_country4report['final type'] == 'mkr')|(risk_country4report['final type'] == 'kpr')|(risk_country4report['final type'] == 'kdr') ,'הכספים נשלחו ל'+risk_country4report['risk_content'] + ' ,מדינה המוגדרת בסיכון גבוה.','')




final_client2report = risk_country4report[['risk_full_content','Content','Title','עברית_dest','עברית','report_name','report_id','final type','Cust ID NO','Cust Address','Cust City','Cust DOB','Sender' ]]

#final_client2report = final_client2report.merge(check_sent[['Sender','Rec Agent Branch Name']], on = 'Sender', how = 'left')
#final_client2report = final_client2report.merge(Branches[['Отделение','סעיף 5']],left_on = 'Rec Agent Branch Name',right_on = 'Отделение',how = 'left')
#final_client2report = final_client2report.rename(columns = {'Rec Agent Branch Name':'final agent'})



final_client2report = final_client2report.fillna('')
final_client2report = final_client2report.drop_duplicates('Sender')
final_client2report.to_excel("monitoring.xlsx")

filepath_check = transction_folder_path+"\*טמפלייט*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    template = textfile
document = MailMerge(template)
print(document.get_merge_fields())



for i in final_client2report.index:
     #template1 = 'test1.docx'
    document = MailMerge(template)
    document.merge(
        Sender = str(final_client2report['Sender'][i]),
        
        Cust_DOB = str(final_client2report['Cust DOB'][i]),
        Title = str(final_client2report['Title'][i]),
        report_id = str(final_client2report['report_id'][i]),
        Content = str(final_client2report['Content'][i]),
        risk_full_content = str(final_client2report['risk_full_content'][i]),
        עברית = str(final_client2report['עברית'][i]),
        Cust_City = str(final_client2report['Cust City'][i]),
        Cust_ID_NO = str(final_client2report['Cust ID NO'][i]),
        Cust_Address = str(final_client2report['Cust Address'][i]),
       
        )
    

    output =  str(final_client2report['report_name'][i]) + "-" + str(final_client2report['report_id'][i]) + '.docx'
    document.write(output)

#filtering outbound reports
sent_report = final_client2report[(final_client2report['final type'] == 'p') |(final_client2report['final type'] == 'd') |(final_client2report['final type'] == 'dr') |(final_client2report['final type'] == 'pr') | (final_client2report['final type'] == 'ppr')]

#creating a data frames with only need to report transctions
sent_report_tr = check_sent[check_sent['Sender'].isin(sent_report['Sender'])]

#leaving only the imporatant columns
sent_report_tr = sent_report_tr[['Order No','TX Date','Ben Country','Sender','Receiver','Amount','Currency','Cust ID NO','nis sum']]

#creating an excel shhet with the ountbound reports
i=0
for_word_table=[]
xlsx=pd.ExcelWriter("monitoring.xlsx", engine='openpyxl', mode='a',if_sheet_exists='overlay')
#round the sum in ils
#sent_report_tr["nis sum"]=sent_report_tr["nis sum"].round()

#creating a unique list of the sender names
ls_customer=sent_report_tr["Sender"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer:
    df=sent_report_tr[sent_report_tr["Sender"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Sender", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=i,index=False)
    for_word_table.append(orderd)
    i=i+len(orderd)+2
            
    xlsx.save()
    
#indexing the outbound_report data frame
info2r = sent_report.set_index('Sender')
info2r_nona = info2r[info2r.index.notnull()]

for title in ls_customer:
    
    data = sent_report_tr[sent_report_tr['Sender'] == title].round()
    table = [["סהכ",data["nis sum"].sum().round(),data['Amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["Sender", "nis sum",'Amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(' ')
    
    orderd=change_names_and_order(transction_folder_path+r"/change_names.xlsx",orderd)
    
    orderd['תאריך פעולה'] = orderd['תאריך פעולה'].astype(str)
    
    doc = docx.Document(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])
                        
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
    
    doc.save(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")

#filtering only shared owner reports
sent_report_ow = final_client2report[(final_client2report['final type'] == 'm') |(final_client2report['final type'] == 'mr')]

#concatacating the two kinds of outbound shared owner transction dataframes
sent_tr_ow = pd.concat([check_sent_ow_risk,check_sent_ow])

#filtering only need to report transctions
sent_report_tr_ow = check_sent[check_sent['Receiver'].isin(sent_tr_ow['Receiver'])]

#leaving only the imporatant columns
sent_report_tr_ow = sent_report_tr_ow[['Order No','TX Date','Ben Country','Sender','Receiver','Amount','Currency','Cust ID NO','nis sum']]

#creating a new list
j=0
for_word_table_ow=[]

#wrting an excel file containing the report transctions for each customer
xlsx=pd.ExcelWriter("monitoring.xlsx", engine='openpyxl', mode='a',if_sheet_exists='overlay')

#round the sum in ils
sent_report_tr_ow["nis sum"]=sent_report_tr_ow["nis sum"].round()

#creating a unique list of the sender names
ls_customer_ow=sent_report_tr_ow["Receiver"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer_ow:
    df=sent_report_tr_ow[sent_report_tr_ow["Receiver"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Sender", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='ow_Reports',startrow=j,index=False)
    for_word_table_ow.append(orderd)
    j=j+len(orderd)+2
            
    xlsx.save()
    
#creating a list containing the client name and owner name
info2r_ow = sent_report_ow[['report_name','report_id','Sender']].merge(sent_report_tr_ow[['Sender','Receiver']], on = 'Sender', how = 'left')
info2r_ow = info2r_ow.drop_duplicates('Receiver')
info2r_ow = info2r_ow.set_index('Receiver')


for title in ls_customer_ow:    
    data = sent_report_tr_ow[sent_report_tr_ow['Receiver'] == title]
    table = [["סהכ",data["nis sum"].sum().round(),data['Amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["Receiver", "nis sum",'Amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(" ")
    orderd=change_names_and_order(transction_folder_path+r"/change_names.xlsx",orderd)
    
    doc = docx.Document(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])                   
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
   
    doc.save(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")
    
    
#creating an excel filies for monitoring the results
monitoring_name_excel = sent_monitoring_excel.merge(final_client2report[['Sender','final type']], on = 'Sender', how = 'left')
monitoring_name_excel['status'] = ''
for i in monitoring_name_excel.index:
        
    if monitoring_name_excel['nis sum'][i] <=47000 :
        monitoring_name_excel['status'][i] = 'סך עסקאות נמוך מרף הדיווח'

    elif monitoring_name_excel['Order No'][i] <= 1 :
        monitoring_name_excel['status'][i] = 'הועבר דיווח רגיל'
    else :
        monitoring_name_excel['status'][i] = monitoring_name_excel['final type'][i]
        
monitoring_name_excel = monitoring_name_excel.sort_values('nis sum', ascending = False)
monitoring_name_excel['final type'] = monitoring_name_excel['final type'].fillna(monitoring_name_excel['status'])
monitoring_name_excel = monitoring_name_excel.rename(columns= {'final type':'final status'})
monitoring_name_excel['final status'] = monitoring_name_excel['final status'].replace({'pr':'פיצול בסיכון','p':'פיצול','dr':'דירדוס בסיכון','d':'דירדוס','m':'מושך משותף','mr':'מושך משותף בסיכון'})

#xlsx = pd.ExcelWriter('monitoring.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay')
xlsx=pd.ExcelWriter('monitoring.xlsx', engine='openpyxl', mode='a',if_sheet_exists='overlay')
monitoring_name_excel[['Sender','nis sum','Order No','final status','month']].to_excel(xlsx, sheet_name='ניטור',index=False)
xlsx.save()


{'עברית', 'Cust_ID_NO', 'Sender', 'Cust_City', 'Content', 'report_id', 'risk_full_content', 'Cust_Address', 'Cust_DOB', 'Title'}


In [37]:
#getting the data ready
#naming rates columns
rates.columns=["curr","rate"]

#adding the rates value
#check_paid = check_paid[check_paid['Canceled Time'].isna()]
check_paid= check_paid.merge(rates, how = 'left', left_on = 'Currency', right_on = 'curr')
check_paid['nis sum'] = check_paid['rate'] * check_paid['Amount']
check_paid = check_paid.dropna(subset = 'nis sum')
check_paid['nis sum'] = check_paid['nis sum'].astype(int)

#lower the country cases:
check_paid['Ben IDIssued By Country'] = check_paid['Ben IDIssued By Country'].str.lower()
check_paid['Cust Country'] = check_paid['Cust Country'].str.lower()
#check['Source country'] = check['Source country'].str.lower()
rc['Alpha-2 code'] = rc['Alpha-2 code'].str.lower()
check_paid = check_paid.fillna('')

#risk check
#check_sent = check.merge(rc[['English short name (using title case)','סיכון/ללא דיווח רגיל']] , left_on = 'Source country', right_on = 'English short name (using title case)',how = 'left',suffixes = ('','_source'))
check_paid = check_paid.merge(rc[['Alpha-2 code','סיכון/ללא דיווח רגיל']] , left_on = 'Cust Country', right_on = 'Alpha-2 code',how = 'left',suffixes = ('','_dest'))
check_paid = check_paid.merge(rc[['Alpha-2 code','סיכון/ללא דיווח רגיל']] , left_on = 'Ben IDIssued By Country', right_on = 'Alpha-2 code',how = 'left',suffixes = ('','_issuance'))

#outbound or inbound
#outbound = check[check['Type'] == 'SEND']
#inbound = check[check['Type'] == 'PAYOUT']

#date coolums as dat and time and a columns with the month number
check_paid['TX Date']= pd.to_datetime(check_paid['TX Date'],dayfirst=True)
check_paid['month'] = pd.DatetimeIndex(check_paid['TX Date']).month

#getting the names ready
check_paid['Receiver'] = (check_paid['Customer Name First'] + ' ' + check_paid['Customer Name Last 1']).str.lower()
check_paid['Sender'] = (check_paid['Ben First Name'] + ' ' + check_paid['Ben Last Name 1']).str.lower()

#check risk 
check_risk_paid = check_paid[ (check_paid['סיכון/ללא דיווח רגיל'].notna()) | (check_paid['סיכון/ללא דיווח רגיל_issuance'].notna())]

#building the monitoring df with sum per client, deal distinct count and under 50k deal count
paid_monitoring = check_paid.groupby(['Sender','month']).agg({'nis sum':'sum','Order No':'nunique'})
paid_monitoring = paid_monitoring.reset_index()
paid_monitoring_excel = paid_monitoring
#sorting the values by sum
paid_monitoring = paid_monitoring.sort_values('nis sum',ascending = False)

#filter only client over 47K
paid_monitoring = paid_monitoring[paid_monitoring['nis sum'] >= 47000]

#adding a column named status based on unique count of deal number
paid_monitoring = paid_monitoring[((paid_monitoring['Order No'] > 1) & (paid_monitoring['nis sum'] >= 50000))|(paid_monitoring['nis sum'] < 50000)]

#marking dirdos transction as d in a new column named dirdos
paid_monitoring['type'] = np.where((paid_monitoring['nis sum']>= 47000) & (paid_monitoring['nis sum']<50000) , "d",'p')
paid_monitoring=paid_monitoring.reset_index()

#out bound risk
#check_sent_risk = check_sent[(check_sent['סיכון/ללא דיווח רגיל'].notna()) | (check_sent['סיכון/ללא דיווח רגיל_dest'].notna()) | (check_sent['סיכון/ללא דיווח רגיל_issuance'].notna())]

#outbound tisk monitorng df
paid_risk_monitoring = check_risk_paid.groupby(['Sender','month']).agg({'nis sum':'sum','Order No':'nunique'})

#sorting the values by sum
paid_risk_monitoring = paid_risk_monitoring.sort_values('nis sum',ascending = False)

#filtering to report tr
paid_risk_monitoring = paid_risk_monitoring[paid_risk_monitoring['nis sum'] >= 4700]
paid_risk_monitoring = paid_risk_monitoring[((paid_risk_monitoring['Order No'] > 1) & (paid_risk_monitoring['nis sum'] >= 5000)) | (paid_risk_monitoring['nis sum'] < 5000)]


#adding the type of the report pr-pitzul dr-dirdos (risk)
paid_risk_monitoring['type'] =  np.where((paid_risk_monitoring['nis sum'] >= 5000), "pr" , "dr")
paid_risk_monitoring = paid_risk_monitoring.reset_index()

try:
    paid_report = paid_risk_monitoring[['type','Sender']].merge(paid_monitoring[['type','Sender']], on = 'Sender',how = 'outer',suffixes = ('_risk',''))
except KeyError:
    report = paid_risk_monitoring[['type','Sender']]
    print('no no risk reports')
    
#building the monitoring df with sum per client and deal distinct count
check_paid_ow = check_paid.groupby(['Receiver','month']).agg({'nis sum':'sum','Order No':'nunique','Sender':'nunique'})

#sorting the values by sum
check_paid_ow = check_paid_ow.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
check_paid_ow = check_paid_ow[(check_paid_ow['Order No'] > 1) & (check_paid_ow['Sender'] > 1) & (check_paid_ow['nis sum'] >= 50000)]
check_paid_ow = check_paid_ow.reset_index()

#creating a list of the shared owner report clients name
check_paid_ow = check_paid_ow.merge(check_paid_ow[['Sender','Receiver']], on = 'Receiver', how = 'left',suffixes = ('_count','') )
check_paid_ow = check_paid_ow.merge(paid_report, on ='Sender' , how = 'left', suffixes = ('','_report'))


try:
    check_paid_ow = check_paid_ow[ (check_paid_ow['type'].isna()) & (check_paid_ow['type_risk'].isna())]
except KeyError:
    check_paid_ow =  check_paid_ow[(check_paid_ow['type_risk'].isna())]
        
check_paid_ow['type_m'] = 'm'
check_paid_ow = check_paid_ow.drop_duplicates('Receiver')
try:
    paid_report = check_paid_ow[['Sender','type_m']].merge(paid_report, on = 'Sender', how = 'outer')
except KeyError:
    paid_report = paid_report
    print('no shared owner reports')    
    
#building the monitoring df with sum per client and deal distinct count risk
check_paid_ow_risk = check_risk_paid.groupby(['Receiver','month']).agg({'nis sum':'sum','Order No':'nunique','Sender':'nunique'})

#sorting the values by sum
check_paid_ow_risk = check_paid_ow_risk.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
check_paid_ow_risk = check_paid_ow_risk[(check_paid_ow_risk['Order No'] > 1) & (check_paid_ow_risk['Sender'] > 1) & (check_paid_ow_risk['nis sum'] >= 5000)]
check_paid_ow_risk = check_paid_ow_risk.reset_index()

#creating a list of the shared owner report clients name____not ready yet!!!!
check_paid_ow_risk = check_paid_ow_risk.merge(check_risk_paid[['Sender','Receiver']], on = 'Receiver', how = 'left',suffixes = ('_count','') )
check_paid_ow_risk = check_paid_ow_risk.merge(paid_report, on ='Sender' , how = 'left')

try:
    check_paid_ow_risk = check_paid_ow_risk[(check_paid_ow_risk['type'].isna()) & (check_paid_ow_risk['type_risk'].isna()) & (check_paid_ow_risk['type_m'].isna())]
except KeyError:
    check_paid_ow_risk =  check_paid_ow_risk

check_paid_ow_risk['type_m_risk'] = 'mr'
check_paid_ow_risk = check_paid_ow_risk.drop_duplicates('Receiver')

#merging the risk ow with regular report
try:
    paid_report = check_paid_ow_risk[['Sender','type_m_risk']].merge(paid_report, on = 'Sender', how = 'outer')
except KeyError:
    paid_report = paid_report

#creating the details to report inbound and outbound
#outbound
paid_report = paid_report.merge(check_paid[['Sender','Cust Country','Receiver','Ben DOB','Ben City','Ben Address','Ben Id No','Ben IDIssued By Country']],on = 'Sender',how = 'left')
paid_report = paid_report.drop_duplicates('Sender')
paid_report = paid_report.fillna('')
paid_report['final type'] = paid_report['type']+paid_report['type_risk']+paid_report['type_m'] + paid_report['type_m_risk']

paid_full_report = paid_report.fillna('')

#importing the report number needed
report_id = (report_num.iloc[1,5] + 1).astype(int)
paid_full_report['report_id'] = report_id + range(len(paid_full_report.index))
paid_full_report['report_name'] = report_num.iloc[3,1].rsplit('-',1)[0]

#checking the risk type
risk_country4report = paid_full_report.merge(rc[['Alpha-2 code','עברית','סיכון/ללא דיווח רגיל']], left_on = 'Ben IDIssued By Country', right_on ='Alpha-2 code', how = 'left')
risk_country4report = risk_country4report.merge(rc[['Alpha-2 code','עברית','סיכון/ללא דיווח רגיל']], left_on = 'Cust Country', right_on ='Alpha-2 code', how = 'left',suffixes = ('','_dest'))

#creating a column with the hebrew name of the country for risk report
risk_country4report['risk_content'] = np.where((risk_country4report['סיכון/ללא דיווח רגיל'].notna()) & ((risk_country4report['final type'] != 'm') | (risk_country4report['final type'] != 'p') | (risk_country4report['final type'] != 'd')) ,risk_country4report['עברית'],risk_country4report['עברית_dest'])

#merging the report content
risk_country4report = risk_country4report.merge(content, left_on = 'final type' , right_on = 'type', how = 'left')
risk_country4report['risk_full_content'] ='הכספים נשלחו ל'+risk_country4report['risk_content'] + ' ,מדינה המוגדרת בסיכון גבוה.'

risk_country4report['risk_full_content'] =np.where((risk_country4report['final type'] == 'mr')|(risk_country4report['final type'] == 'pr')|(risk_country4report['final type'] == 'dr')|(risk_country4report['final type'] == 'mkr')|(risk_country4report['final type'] == 'kpr')|(risk_country4report['final type'] == 'kdr') ,'הכספים נשלחו ל'+risk_country4report['risk_content'] + ' ,מדינה המוגדרת בסיכון גבוה.','')

final_client2report = risk_country4report[['risk_full_content','Content','Title','עברית_dest','עברית','report_name','report_id','final type','Ben Id No','Ben Address','Ben City','Ben DOB','Sender' ]]

final_client2report = final_client2report.fillna('')
final_client2report.to_excel("monitoring_paid.xlsx")

filepath_check = transction_folder_path+"\*טמפלייט*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    template = textfile
document = MailMerge(template)
print(document.get_merge_fields())


for i in final_client2report.index:
     #template1 = 'test1.docx'
    document = MailMerge(template)
    document.merge(
        Sender = str(final_client2report['Sender'][i]),
        
        Cust_DOB = str(final_client2report['Ben DOB'][i]),
        Title = str(final_client2report['Title'][i]),
        report_id = str(final_client2report['report_id'][i]),
        Content = str(final_client2report['Content'][i]),
        risk_full_content = str(final_client2report['risk_full_content'][i]),
        עברית = str(final_client2report['עברית'][i]),
        Cust_City = str(final_client2report['Ben City'][i]),
        Cust_ID_NO = str(final_client2report['Ben Id No'][i]),
        Cust_Address = str(final_client2report['Ben Address'][i]),
        )

    output =  str(final_client2report['report_name'][i]) + "-" + str(final_client2report['report_id'][i]) + '.docx'
    document.write(output)

#filtering outbound reports
sent_report = final_client2report[(final_client2report['final type'] == 'p') |(final_client2report['final type'] == 'd') |(final_client2report['final type'] == 'dr') |(final_client2report['final type'] == 'pr') | (final_client2report['final type'] == 'ppr')]

#creating a data frames with only need to report transctions
sent_report_tr = check_paid[check_paid['Sender'].isin(paid_report['Sender'])]

#leaving only the imporatant columns
sent_report_tr = sent_report_tr[['Order No','TX Date','Cust Country','Sender','Receiver','Amount','Currency','Ben Id No','nis sum']]

#creating an excel shhet with the ountbound reports
i=0
for_word_table=[]
xlsx=pd.ExcelWriter("monitoring_paid.xlsx", engine='openpyxl', mode='a',if_sheet_exists='overlay')
#round the sum in ils
sent_report_tr["nis sum"]=sent_report_tr["nis sum"].round()

#creating a unique list of the sender names
ls_customer=sent_report_tr["Sender"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer:
    df=sent_report_tr[sent_report_tr["Sender"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Sender", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=i,index=False)
    for_word_table.append(orderd)
    i=i+len(orderd)+2
            
    xlsx.save()
    
#indexing the outbound_report data frame
info2r = sent_report.set_index('Sender')
info2r_nona = info2r[info2r.index.notnull()]

for title in ls_customer:
    
    data = sent_report_tr[sent_report_tr['Sender'] == title].round()
    table = [["סהכ",data["nis sum"].sum().round(),data['Amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["Sender", "nis sum",'Amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(' ')
    
    orderd=change_names_and_order(transction_folder_path+r"/change_names_paid.xlsx",orderd)
    
    orderd['תאריך פעולה'] = orderd['תאריך פעולה'].astype(str)
    
    doc = docx.Document(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])
                        
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
    
    doc.save(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")

#filtering only shared owner reports
sent_report_ow = final_client2report[(final_client2report['final type'] == 'm') |(final_client2report['final type'] == 'mr')]

#concatacating the two kinds of outbound shared owner transction dataframes
sent_tr_ow = pd.concat([check_paid_ow_risk,check_paid_ow])

#filtering only need to report transctions
sent_report_tr_ow = check_paid[check_paid['Receiver'].isin(sent_tr_ow['Receiver'])]

#leaving only the imporatant columns
sent_report_tr_ow = sent_report_tr_ow[['Order No','TX Date','Cust Country','Sender','Receiver','Amount','Currency','Ben Id No','nis sum']]

#creating a new list
j=0
for_word_table_ow=[]

#wrting an excel file containing the report transctions for each customer
xlsx=pd.ExcelWriter("monitoring_paid.xlsx", engine='openpyxl', mode='a',if_sheet_exists='overlay')

#round the sum in ils
sent_report_tr_ow["nis sum"]=sent_report_tr_ow["nis sum"].round()

#creating a unique list of the sender names
ls_customer_ow=sent_report_tr_ow["Receiver"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer_ow:
    df=sent_report_tr_ow[sent_report_tr_ow["Receiver"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Sender", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=j,index=False)
    for_word_table_ow.append(orderd)
    j=j+len(orderd)+2
            
    xlsx.save()
    
#creating a list containing the client name and owner name
info2r_ow = sent_report_ow[['report_name','report_id','Sender']].merge(sent_report_tr_ow[['Sender','Receiver']], on = 'Sender', how = 'left')
info2r_ow = info2r_ow.drop_duplicates('Receiver')
info2r_ow = info2r_ow.set_index('Receiver')


for title in ls_customer_ow:    
    data = sent_report_tr_ow[sent_report_tr_ow['Receiver'] == title]
    table = [["סהכ",data["nis sum"].sum().round(),data['Amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["Receiver", "nis sum",'Amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(" ")
    orderd=change_names_and_order(transction_folder_path+r"/change_names_paid.xlsx",orderd)
    
    doc = docx.Document(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])                   
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
   
    doc.save(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")
    

{'עברית', 'Cust_ID_NO', 'Sender', 'Cust_City', 'Content', 'report_id', 'risk_full_content', 'Cust_Address', 'Cust_DOB', 'Title'}


In [38]:
#creating an excel filies for monitoring the results
monitoring_name_excel = paid_monitoring_excel.merge(final_client2report[['Sender','final type']], on = 'Sender', how = 'left')
monitoring_name_excel['status'] = ''
for i in monitoring_name_excel.index:
        
    if monitoring_name_excel['nis sum'][i] <=47000 :
        monitoring_name_excel['status'][i] = 'סך עסקאות נמוך מרף הדיווח'

    elif monitoring_name_excel['Order No'][i] <= 1 :
        monitoring_name_excel['status'][i] = 'הועבר דיווח רגיל'
    else :
        monitoring_name_excel['status'][i] = monitoring_name_excel['final type'][i]
        
monitoring_name_excel = monitoring_name_excel.sort_values('nis sum', ascending = False)
monitoring_name_excel['final type'] = monitoring_name_excel['final type'].fillna(monitoring_name_excel['status'])
monitoring_name_excel = monitoring_name_excel.rename(columns= {'final type':'final status'})
monitoring_name_excel['final status'] = monitoring_name_excel['final status'].replace({'pr':'פיצול בסיכון','p':'פיצול','dr':'דירדוס בסיכון','d':'דירדוס','m':'מושך משותף','mr':'מושך משותף בסיכון'})

#xlsx = pd.ExcelWriter('monitoring.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay')
xlsx=pd.ExcelWriter('monitoring_paid.xlsx', engine='openpyxl', mode='a',if_sheet_exists='overlay')
monitoring_name_excel[['Sender','nis sum','Order No','final status','month']].to_excel(xlsx, sheet_name='ניטור',index=False)
xlsx.save()
